In [7]:
import numpy as np
from torchsummary import summary
import pickle
import torch
import datetime
from torch.utils.tensorboard import SummaryWriter

In [8]:
f=open('./FFT_AllSubject_Training_Face_minmaxNorm','rb')
fftdata=pickle.load(f)

In [9]:
result = np.zeros((1,3,102,188))
for l in range(len(fftdata)):
    fftdata[l]=np.where(fftdata[l]>1,1,fftdata[l])
    #totalsample+=fftdata[i].shape[0]
    result = np.vstack((result,fftdata[l]))

In [10]:
class VAE(torch.nn.Module):
    def __init__(self,k_size,channel):
        super(VAE, self).__init__()
        self.device=None
        if torch.cuda.is_available():
            self.device = torch.device('cuda')
        else:
            self.device = torch.device('cpu')
        
        # input dim 3*102*188 = 57528
        # H = [ (HIn + 2×padding[0]−dilation[0]×(kernel_size[0]−1)−1)/s ] + 1
        #   =    [( 102 + 0 - 1x(2) -1 )/2] + 1 
        # W = [ (WIn +2×padding[0]−dilation[0]×(kernel_size[0]−1)−1)/s ] + 1
        #   = [ ( 188 + 0 - 1x(2) -1 ) /2 ] + 1
        # ConvTranspose 
        # Out =(In−1)×stride[0]−2×padding[0]+dilation[0]×(kernel_size[0]−1)+output_padding[0]+1
        #     = 4x2 - 0 + 2 + 0 + 1 = 11
        #     = 9x2 - 0 + 2 + 0 + 1 = 21
        self.activation = torch.nn.ReLU()
        self.maxpool = torch.nn.MaxPool2d(3, stride=2)
        
        self.encoder =  torch.nn.Sequential(
            torch.nn.Conv2d(channel, 64, k_size, stride=1,padding=1),  #  C=64,H=102,W=188
            torch.nn.Conv2d(64, 64, k_size, stride=1,padding=1),  #  C=64,H=102,W=188
            self.activation,
            self.maxpool,                                         #  C=64,H=50,W=93
            torch.nn.Conv2d(64, 128, k_size,stride=1,padding=1),   #  C=128,H=50,W=93
            torch.nn.Conv2d(128, 128, k_size,stride=1,padding=1),  #  C=128,H=50,W=93
            torch.nn.Conv2d(128, 128, k_size,stride=1,padding=1),  #  C=128,H=50,W=93
            self.activation,
            self.maxpool,                                          #  C=128,H=24,W=46
            torch.nn.Conv2d(128, 256, k_size,stride=1,padding=1),  #  C=256,H=24,W=46
            torch.nn.Conv2d(256, 256, k_size,stride=1,padding=1),   #  C=256,H=24,W=46
            torch.nn.Conv2d(256, 256, k_size,stride=1,padding=1),   #  C=256,H=24,W=46
            self.activation,
            self.maxpool,                                          #  C=256,H=11,W=22
            torch.nn.Conv2d(256, 512, k_size,stride=1,padding=1),  #  C=512,H=11,W=22
            torch.nn.Conv2d(512, 512, k_size,stride=1,padding=1),   #  C=512,H=11,W=22
            torch.nn.Conv2d(512, 512, k_size,stride=1,padding=1),   #  C=512,H=11,W=22
            self.activation,
            self.maxpool                                          #  C=512,H=5,W=10 = 25600 - mu C[0:256] , v C[256:512]
        ).to(self.device)
        self.decoder =  torch.nn.Sequential(
            torch.nn.Conv2d(256, 16, k_size, stride=1,padding=1),  #  C=16,H=5,W=10
            torch.nn.Conv2d(16, 32, k_size, stride=1,padding=1),   #  C=32,H=5,W=10
            torch.nn.ConvTranspose2d(32,32,(3,4),stride=2),        #  C=32,H=11,W=22
            self.activation,
            torch.nn.Conv2d(32, 64, k_size,stride=1,padding=1 ),   #  C=64,H=11,W=22
            torch.nn.Conv2d(64, 128, k_size,stride=1,padding=1 ),  # C=128,H=11,W=22
            torch.nn.ConvTranspose2d(128,128,(4,4),stride=2),      # C=128,H=24,W=46
            self.activation,
            torch.nn.Conv2d(128, 256, k_size,stride=1,padding=1),   #  C=64,H=24,W=46
            torch.nn.Conv2d(256, 32, k_size,stride=1,padding=1),    #  C=32,H=24,W=46
            torch.nn.ConvTranspose2d(32,32,(4,3),stride=2),        #  C=32,H=50,W=93
            self.activation,
            torch.nn.Conv2d(32, 16, k_size,stride=1,padding=1),    #  C=16,H=50,W=93
            torch.nn.Conv2d(16, 16, k_size,stride=1,padding=1),    #  C=16,H=50,W=93
            torch.nn.ConvTranspose2d(16,3,(4,4),stride=2),          #  C=3,H=102,W=188
            torch.nn.Sigmoid()
        ).to(self.device)
        
        self.parameters = set()
        self.parameters |= set(self.encoder.parameters())
        self.parameters |= set(self.decoder.parameters())
        self.optimizer = torch.optim.Adam(self.parameters, lr=0.001)
        
    def reparametrize(self,mu,log_var):
        #Reparametrization Trick to allow gradients to backpropagate from the 
        #stochastic part of the model
        sigma = torch.exp(0.5*log_var)
        z = torch.randn(mu.size(0),mu.size(1),mu.size(2),mu.size(3),device=self.device)
        z= z.type_as(mu)
        return mu + sigma*z
        
    def forward(self, x):
        out = self.encoder(x)
        mu = out[:,:256,:,:]
        v = out[:,256:,:,:]
        #print(mu)
        #print(v)
        out = self.reparametrize(mu,v)
        out = self.decoder(out)
        #print(mu)
        return out,mu,v

In [27]:
coder = VAE(3,3)
#summary(coder,(3,102,188))
## Training 
bceloss=torch.nn.BCELoss()
#print(fftdata[1][111,0,0])
epoch=100
batchsize=10

tb = SummaryWriter('Training_CNS/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'_R2_100epoch')
length=[x for x in range(result.shape[0])]
t=int(np.floor(result.shape[0]*0.7))
np.random.shuffle(length)
print(t)
tidxs=length[:t]
vidxs=length[t:]
#print(tidxs)
batch=t//10
vbatch=(result.shape[0]-t)//10
bestvloss=float('inf')
for e in range(epoch):
    trainingloss=0
    rloss=0
    kloss=0
    vloss=0
    vrloss=0
    vkloss=0
    coder.train()
    for i in range(batch):
        idx=tidxs[i*batchsize:i*batchsize+batchsize]
        #print(idx)
        data = torch.tensor(result[idx,:,:,:]).float().to(coder.device)
        out,mu,v = coder(data)
        regconstructionloss = bceloss(out,data)
        KLDloss = - 0.5 * torch.sum(1+ v - mu.pow(2) - v.exp())
            #print(f'R {regconstructionloss}')
            #print(f'K {KLDloss}')
        totalloss=regconstructionloss+KLDloss
        coder.optimizer.zero_grad()
        trainingloss+=totalloss.item()
        rloss+=regconstructionloss.item()
        kloss+=KLDloss.item()
        totalloss.backward()
        coder.optimizer.step()
        
    coder.eval()
    for i in range(vbatch):
        idx=vidxs[i*batchsize:i*batchsize+batchsize]
        #print(idx)
        data = torch.tensor(result[idx,:,:,:]).float().to(coder.device)
        out,mu,v = coder(data)
        regconstructionloss = bceloss(out,data)
        KLDloss = - 0.5 * torch.sum(1+ v - mu.pow(2) - v.exp())
            #print(f'R {regconstructionloss}')
            #print(f'K {KLDloss}')
        loss=regconstructionloss+KLDloss
        vloss+=loss.item()
        vrloss+=regconstructionloss.item()
        vkloss+=KLDloss.item()
        #print(totalloss.item())
    if(vloss<bestvloss):
        torch.save(coder,'./Model_CNS/bmodel_'+str(vloss/vbatch))
        bestvloss=vloss
        
    tb.add_scalar("Training Avg Loss", (trainingloss/batch), e)
    tb.add_scalar("Training Avg RLoss", (rloss/batch), e)
    tb.add_scalar("Training Avg KLoss", (kloss/batch), e)
    tb.add_scalar("Validation Avg Loss", (vloss/vbatch), e)
    tb.add_scalar("Validation Avg RLoss", (vrloss/vbatch), e)
    tb.add_scalar("Validation Avg KLoss", (vkloss/vbatch), e)
   
    for name, weight in coder.encoder.named_parameters():
        tb.add_histogram('encoder'+name,weight, e)
        tb.add_histogram(f'encoder_{name}.grad',weight.grad, e)
    for name, weight in coder.decoder.named_parameters():
        tb.add_histogram('decoder'+name,weight, e)
        tb.add_histogram(f'decoder_{name}.grad',weight.grad, e)
    print(f'epoch: {e:3} batch: {i:3} Training Batch Avg loss: {trainingloss/batch:10.8f} Validation Batch Avg loss: {vloss/vbatch:10.8f}')

3295
epoch:   0 batch: 140 Training Batch Avg loss: 160199027488709.96875000 Validation Batch Avg loss: 0.13245991
epoch:   1 batch: 140 Training Batch Avg loss: 0.13140317 Validation Batch Avg loss: 0.12948090
epoch:   2 batch: 140 Training Batch Avg loss: 0.12920577 Validation Batch Avg loss: 0.12753223
epoch:   3 batch: 140 Training Batch Avg loss: 0.12712301 Validation Batch Avg loss: 0.12532974
epoch:   4 batch: 140 Training Batch Avg loss: 0.12496145 Validation Batch Avg loss: 0.12372442
epoch:   5 batch: 140 Training Batch Avg loss: 0.12385029 Validation Batch Avg loss: 0.12244790
epoch:   6 batch: 140 Training Batch Avg loss: 0.12305862 Validation Batch Avg loss: 0.12178563
epoch:   7 batch: 140 Training Batch Avg loss: 0.12239055 Validation Batch Avg loss: 0.12105080
epoch:   8 batch: 140 Training Batch Avg loss: 0.12188128 Validation Batch Avg loss: 0.12070738
epoch:   9 batch: 140 Training Batch Avg loss: 0.12145520 Validation Batch Avg loss: 0.12040316
epoch:  10 batch: 140

In [87]:
fftdata1 = np.asarray(fftdata, dtype=np.float32)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (16,) + inhomogeneous part.

In [60]:
np.where(fftdata[1]<0)

(array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64))